**IBM Applied Data Science Capstone Course by Coursera**

**Starting a New Multiplex in Hyderabad, India**

**Importing Required Libraries**

In [1]:
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
import json
!pip install geocoder
from geopy.geocoders import Nominatim
import geocoder 
import requests 
from bs4 import BeautifulSoup 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 
print("Libraries imported.")

    100% |████████████████████████████████| 102kB 17.1MB/s 
Libraries imported.


**Scrape data from Wiki page to a DataFrame**

In [2]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Hyderabad,_India").text
soup = BeautifulSoup(data, 'html.parser')
neighborhoodList = []
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)
hy_df = pd.DataFrame({"Neighborhood": neighborhoodList})
hy_df.head()

,Neighborhood
0,A. S. Rao Nagar
1,A.C. Guards
2,Abhyudaya Nagar
3,Abids
4,Adikmet


**Geographical Co-ordinates**

In [10]:
def get_latlng(neighborhood):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{},Hyderabad,India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [11]:
coords = [ get_latlng(neighborhood) for neighborhood in hy_df["Neighborhood"].tolist() ]
coords

[[17.394870000000026, 78.47076000000004],
 [17.397519158432747, 78.45998615301222],
 [17.33755000000002, 78.56235000000004],
 [17.389800000000037, 78.47658000000007],
 [17.408450000000073, 78.51196000000004],
 [17.377450000000067, 78.47736000000003],
 [17.383473460582927, 78.46606544334597],
 [17.346310000000074, 78.47241000000008],
 [17.362240000000043, 78.48157000000003],
 [17.51908000000003, 78.41435000000007],
 [17.535430000000076, 78.54427000000004],
 [17.39247000000006, 78.51944000000003],
 [17.435340000000053, 78.44861000000003],
 [17.408880000000067, 78.49314000000004],
 [17.394480000000044, 78.44457000000006],
 [17.369170000000054, 78.43683000000004],
 [17.411960000000022, 78.50276000000008],
 [17.372720000000072, 78.49047000000007],
 [17.386660000000063, 78.48681000000005],
 [17.38823000000002, 78.49273000000005],
 [17.341825500000006, 78.54753600000001],
 [17.448470000000043, 78.44937000000004],
 [17.415050000000065, 78.43432000000007],
 [17.386669553198413, 78.4782764758919

In [12]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
hy_df['Latitude'] = df_coords['Latitude']
hy_df['Longitude'] = df_coords['Longitude']
print(hy_df.shape)
hy_df

(200, 3)


,Neighborhood,Latitude,Longitude
0,A. S. Rao Nagar,17.394870,78.470760
1,A.C. Guards,17.397519,78.459986
2,Abhyudaya Nagar,17.337550,78.562350
3,Abids,17.389800,78.476580
4,Adikmet,17.408450,78.511960
5,Afzal Gunj,17.377450,78.477360
6,Aghapura,17.383473,78.466065
7,"Aliabad, Hyderabad",17.346310,78.472410
8,Alijah Kotla,17.362240,78.481570
9,Allwyn Colony,17.519080,78.414350


In [13]:
hy_df.to_csv("hy_df.csv", index=False)

**Hyderabad map with Neighborhoods pointed on it**

In [14]:
address = 'Hyderabad,India'
geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Hyderabad,India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Hyderabad,India 17.3616079, 78.4746286.


In [15]:
map_h = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, neighborhood in zip(hy_df['Latitude'], hy_df['Longitude'], hy_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_h)  
map_h

In [16]:
map_h.save('Hyderabad_Map1.html')

**Exploring Neighborhoods with Foursquare**

In [17]:
CLIENT_ID = 'LEUTMDU4BVVYW1KXKM5LBSUVO5JXM3FQML1SNKN1NDP0AVXN' # your Foursquare ID
CLIENT_SECRET = 'DVTODCL524SCW4WPFHVXSTNSOMEXPTYZB0IHQ3QSSNKOITMV' # your Foursquare Secret
VERSION = '20180605'

In [19]:
radius = 2500
LIMIT = 80
venues = []
for lat, long, neighborhood in zip(hy_df['Latitude'], hy_df['Longitude'], hy_df['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,CLIENT_SECRET,VERSION,lat,long,radius,LIMIT)
    results = requests.get(url).json()["response"]['groups'][0]['items']
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [20]:
venues_df = pd.DataFrame(venues)
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
print(venues_df.shape)
venues_df.head()

(8032, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,A. S. Rao Nagar,17.39487,78.47076,Nizam club,17.403221,78.468729,Lounge
1,A. S. Rao Nagar,17.39487,78.47076,Subhan Bakery,17.392412,78.464712,Bakery
2,A. S. Rao Nagar,17.39487,78.47076,Cafe Bahar,17.399595,78.478566,Indian Restaurant
3,A. S. Rao Nagar,17.39487,78.47076,Cafe Niloufer & Bakers,17.399715,78.462881,Café
4,A. S. Rao Nagar,17.39487,78.47076,Santosh Dabha,17.391209,78.477575,Diner


In [21]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
A. S. Rao Nagar,80,80,80,80,80,80
A.C. Guards,80,80,80,80,80,80
Abhyudaya Nagar,10,10,10,10,10,10
Abids,80,80,80,80,80,80
Adikmet,32,32,32,32,32,32
Afzal Gunj,56,56,56,56,56,56
Aghapura,70,70,70,70,70,70
"Aliabad, Hyderabad",19,19,19,19,19,19
Alijah Kotla,22,22,22,22,22,22


**Finding Unique Category**

In [22]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))
venues_df['VenueCategory'].unique()[:50]

There are 156 uniques categories.


array(['Lounge', 'Bakery', 'Indian Restaurant', 'Café', 'Diner',
       'Juice Bar', 'Hotel', 'South Indian Restaurant', 'Ice Cream Shop',
       'Hyderabadi Restaurant', 'Science Museum', 'Burger Joint',
       'Hotel Bar', 'Dessert Shop', 'Chaat Place', 'Multiplex',
       'Chinese Restaurant', 'Shoe Store', 'Smoke Shop', 'Lake',
       'Mobile Phone Shop', 'Neighborhood', 'Stadium', 'Breakfast Spot',
       'Snack Place', 'Restaurant', 'Fast Food Restaurant',
       'Vegetarian / Vegan Restaurant', 'Performing Arts Venue', 'Food',
       'Bar', 'Park', 'Middle Eastern Restaurant', 'Coffee Shop',
       'Gaming Cafe', 'Historic Site', 'Department Store', 'Pizza Place',
       'Arcade', 'Bistro', 'Scenic Lookout', 'BBQ Joint', 'Pub',
       'Donut Shop', 'Bookstore', 'Italian Restaurant', 'Movie Theater',
       'Shopping Mall', 'History Museum', 'Food Court'], dtype=object)

**Let's Analyze every Neighborhood**

In [23]:
hy_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")
hy_onehot['Neighborhoods'] = venues_df['Neighborhood'] 
fixed_columns = [hy_onehot.columns[-1]] + list(hy_onehot.columns[:-1])
hy_onehot = hy_onehot[fixed_columns]
print(hy_onehot.shape)
hy_onehot.head()

(8032, 157)


,Neighborhoods,ATM,Afghan Restaurant,Airport,American Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,BBQ Joint,Bakery,Bank,Bar,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Buffet,Burger Joint,Bus Station,Business Service,Butcher,Cable Car,Café,Campground,Chaat Place,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Harbor / Marina,Historic Site,History Museum,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hunan Restaurant,Hyderabadi Restaurant,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Juice Bar,Lake,Light Rail Station,Liquor Store,Lounge,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Museum,Neighborhood,New American Restaurant,Nightclub,North Indian Restaurant,Office,Outdoors & Recreation,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Pool,Pub,Rajasthani Restaurant,Residential Building (Apartment / Condo),Restaurant,River,Sandwich Place,Scenic Lookout,Science Museum,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Social Club,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Surf Spot,Tea Room,Temple,Thai Restaurant,Theme Park,Trail,Train Station,Vegetarian / Vegan Restaurant,Wings Joint,Women's Store,Zoo
0,A. S. Rao Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,A. S. Rao Nagar,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,A. S. Rao Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,A. S. Rao Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,A. S. Rao Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [24]:
hy_grouped = hy_onehot.groupby(["Neighborhoods"]).mean().reset_index()
print(hy_grouped.shape)
hy_grouped

(200, 157)


,Neighborhoods,ATM,Afghan Restaurant,Airport,American Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,BBQ Joint,Bakery,Bank,Bar,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Buffet,Burger Joint,Bus Station,Business Service,Butcher,Cable Car,Café,Campground,Chaat Place,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Harbor / Marina,Historic Site,History Museum,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hunan Restaurant,Hyderabadi Restaurant,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Juice Bar,Lake,Light Rail Station,Liquor Store,Lounge,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Museum,Neighborhood,New American Restaurant,Nightclub,North Indian Restaurant,Office,Outdoors & Recreation,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Pool,Pub,Rajasthani Restaurant,Residential Building (Apartment / Condo),Restaurant,River,Sandwich Place,Scenic Lookout,Science Museum,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Social Club,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Surf Spot,Tea Room,Temple,Thai Restaurant,Theme Park,Trail,Train Station,Vegetarian / Vegan Restaurant,Wings Joint,Women's Store,Zoo
0,A. S. Rao Nagar,0.000000,0.000000,0.000000,0.000000,0.012500,0.0000,0.000000,0.000000,0.0000,0.000000,0.050000,0.000000,0.012500,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012500,0.000000,0.000000,0.012500,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.012500,0.037500,0.000000,0.000000,0.012500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012500,0.025000,0.012500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.012500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012500,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000,0.012500,0.000000,0.000000,0.000000,0.050000,0.025000,0.000000,0.025000,0.075000,0.137500,0.000000,0.000000,0.000000,0.0000,0.000000,0.025000,0.012500,0.000000,0.000000,0.012500,0.000000,0.000000,0.000000,0.012500,0.000000,0.012500,0.000000,0.0000,0.0,0.000000,0.000000,0.050000,0.000000,0.012500,0.0000,0.000000,0.0000,0.0000,0.000000,0.025000,0.0000,0.012500,0.000000,0.012500,0.000000,0.0,0.000,0.000000,0.0000,0.000000,0.025000,0.000000,0.000000,0.000000,0.012500,0.012500,0.000000,0.012500,0.012500,0.000000,0.0000,0.037500,0.000000,0.000000,0.000000,0.0,0.012500,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.025000,0.000000,0.000000,0.000000
1,A.C. Guards,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0000,0.012500,0.062500,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.012500,0.012500,0.000000,0.000000,0.012500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.075000,0.000000,0.012500,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012500,0.012500,0.000000,0.012500,0.000000,0.000000,0.

In [26]:
len(hy_grouped[hy_grouped["Multiplex"] > 0])

93

In [27]:
hy_mall = hy_grouped[["Neighborhoods","Multiplex"]]
hy_mall.head()

,Neighborhoods,Multiplex
0,A. S. Rao Nagar,0.0500
1,A.C. Guards,0.0375
2,Abhyudaya Nagar,0.0000
3,Abids,0.0125
4,Adikmet,0.0000


**Neighborhood Clustering**

In [28]:
kclusters = 3
hy_clustering = hy_mall.drop(["Neighborhoods"], 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(hy_clustering)
kmeans.labels_[0:10]

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [29]:
hy_merged = hy_mall.copy()
hy_merged["Cluster Labels"] = kmeans.labels_

In [30]:
hy_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
hy_merged.head()

,Neighborhood,Multiplex,Cluster Labels
0,A. S. Rao Nagar,0.0500,0
1,A.C. Guards,0.0375,0
2,Abhyudaya Nagar,0.0000,1
3,Abids,0.0125,1
4,Adikmet,0.0000,1


In [31]:
hy_merged = hy_merged.join(hy_df.set_index("Neighborhood"), on="Neighborhood")
print(hy_merged.shape)
hy_merged.head()

(200, 5)


,Neighborhood,Multiplex,Cluster Labels,Latitude,Longitude
0,A. S. Rao Nagar,0.0500,0,17.394870,78.470760
1,A.C. Guards,0.0375,0,17.397519,78.459986
2,Abhyudaya Nagar,0.0000,1,17.337550,78.562350
3,Abids,0.0125,1,17.389800,78.476580
4,Adikmet,0.0000,1,17.408450,78.511960


In [32]:
print(hy_merged.shape)
hy_merged.sort_values(["Cluster Labels"], inplace=True)
hy_merged

(200, 5)


,Neighborhood,Multiplex,Cluster Labels,Latitude,Longitude
0,A. S. Rao Nagar,0.050000,0,17.394870,78.470760
162,Pisal Banda,0.050000,0,17.394870,78.470760
43,Chikkadpally,0.027778,0,17.403010,78.497930
45,Chintal Basti,0.050000,0,17.406620,78.456590
50,Dilsukhnagar,0.040000,0,17.368600,78.535150
51,Domalguda,0.062500,0,17.409950,78.482290
153,Neredmet Kakatiya Nagar,0.045455,0,17.476796,78.537658
152,Neredmet,0.100000,0,17.475436,78.522986
108,List of Bazaars of Hyderabad,0.050000,0,17.394870,78.470760
55,Etymology of localities of Hyderabad,0.050000,0,17.394870,78.470760


**Let's Visualize the Clusters**

In [42]:
hmap_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
markers_colors = ["yellow","green","red"]
for lat, lon, poi, cluster in zip(hy_merged['Latitude'], hy_merged['Longitude'], hy_merged['Neighborhood'], hy_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=markers_colors[cluster],
        fill=True,
        fill_color=markers_colors[cluster],
        fill_opacity=0.7).add_to(hmap_clusters)
hmap_clusters

In [43]:
hmap_clusters.save('Hyderabad_Map2.html')

**Let's Examine each Cluster**

**Cluster 0**

In [39]:
hy_merged.loc[hy_merged['Cluster Labels'] == 0]

,Neighborhood,Multiplex,Cluster Labels,Latitude,Longitude
0,A. S. Rao Nagar,0.050000,0,17.394870,78.470760
162,Pisal Banda,0.050000,0,17.394870,78.470760
43,Chikkadpally,0.027778,0,17.403010,78.497930
45,Chintal Basti,0.050000,0,17.406620,78.456590
50,Dilsukhnagar,0.040000,0,17.368600,78.535150
51,Domalguda,0.062500,0,17.409950,78.482290
153,Neredmet Kakatiya Nagar,0.045455,0,17.476796,78.537658
152,Neredmet,0.100000,0,17.475436,78.522986
108,List of Bazaars of Hyderabad,0.050000,0,17.394870,78.470760
55,Etymology of localities of Hyderabad,0.050000,0,17.394870,78.470760


**Cluster 1**

In [44]:
hy_merged.loc[hy_merged['Cluster Labels'] == 1]

,Neighborhood,Multiplex,Cluster Labels,Latitude,Longitude
187,Sanjeeva Reddy Nagar,0.025000,1,17.438390,78.445300
124,Marredpally,0.000000,1,17.451650,78.510790
186,Sanghi Nagar,0.000000,1,17.389640,78.513450
127,Mehdipatnam,0.000000,1,17.392410,78.442180
126,"Mehboob ki Mehendi, Hyderabad",0.000000,1,17.362095,78.470779
123,Manovikas Nagar,0.000000,1,17.465610,78.493270
184,Sainikpuri,0.000000,1,17.475108,78.521899
128,Mettuguda,0.000000,1,17.434100,78.517740
129,"Minister Road, Hyderabad",0.000000,1,17.432638,78.484448
130,Mir Alam Tank,0.000000,1,17.350908,78.452488


**Cluster 2**

In [41]:
hy_merged.loc[hy_merged['Cluster Labels'] == 2]

,Neighborhood,Multiplex,Cluster Labels,Latitude,Longitude
15,Attapur,0.222222,2,17.369170,78.436830
74,Izzat Nagar,0.200000,2,17.492560,78.453440
84,Karmanghat,0.250000,2,17.341267,78.519316
91,Kompally,0.250000,2,17.543570,78.491580
170,Radha Krishna Nagar,0.285714,2,17.365610,78.423480
72,Hyderguda,0.222222,2,17.370390,78.419000
104,Langar Houz,0.333333,2,17.363510,78.419330
38,Champapet,0.200000,2,17.346690,78.513270


**Observation**

Most of Multiplexes in Hyderabad city are found in areas of cluster 2.It is not good to start a new multiplex in neighborhoods of cluster 2
because of very high competition.Neighborhoods in cluster 0 have moderate multiplex numbers but still there will be tough competition.
But Neighborhoods of cluster 1 has less number of multiplex when compared to other clusters.There are many Neighborhoods with out any multiplex,
It is good to start in neighborhoods of cluster 1 as there will be little or no competition.